# Modelo lindo de infecção progressiva estocastica


## Introdução

A alguns dias atrás, vi no facebok a seguinte imagem:

![Modelo de infecção com isolamento em árvore](https://scontent.fcpq4-1.fna.fbcdn.net/v/t1.0-9/91112031_3683857418354413_569726385317216256_o.jpg?_nc_cat=100&_nc_sid=8bfeb9&_nc_ohc=T6sLKt5A6IkAX9CZbrx&_nc_pt=1&_nc_ht=scontent.fcpq4-1.fna&oh=17a20312c67a4307eeaf138076b39d5f&oe=5EA33542)

Ela descreve um modelo de infeção em árvore, onde uma pessoa tem o potencial de infectar outras três. Esse número vem dos relatórios canonicos (?) sobre infectologia, mas representa um modelo muito solto em relação ao cenário real.

Abaixo eu descrevo um modelo que acredito ser mais realista, baseado em relacionamentos extraídos de uma base dados sobre conecções entre usuários do Facebook.

As principais diferenças são:

- Grafo de baixo raio (4.7) e, portanto, muito mais conexo que a árvore acima (aproxima melhor as relações sociais humanas)
- Constantes do sistema são razoavelmente supostas a partir de dados extraídos de fontes estáveis (e.g. IBGE). Algumas destas são: proba home-officing, proba de infecção, dias de quarentena-em-casa.

**Disclaimer:** mesmo que as relações sociais sejam melhor representadas
por grafos complexos de múltiplos *clusters* do que por árvores ou conexões aleatórias, este problema possui uma propriedade fractal onde cada cluster se comporta como um indivíduo.
As mesmas propriedades valem e os eventos são recursivamente passados para os indivíduos membros. Portanto, um modelo de infecção baseado em grafos realistas (como este) e um modelo de infeção aleatório (mais eficiente) se aproximam e possuem
taxas de erro muito similares na prática.

In [ ]:
import os
from math import ceil

import numpy as np
import pandas as pd
from numpy.random import rand
import networkx as nx
from google.colab import drive

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import seaborn as sns

In [ ]:
sns.set()

## Lendo o conjunto de dados *Facebook Circles*

As relações sociais são simuladas pelo grafo descrito no conjunto de dados *Facebook Circles*, contendo links de amizade no Facebook de usuários anonimizados.

Para esta simulação, foi considerada a componente conexa de maior cardinalidade e os demais nós e arestas foram descartados.

In [ ]:
FACEBOOK_DIR = '/content/gdrive/My Drive/datasets/facebook'

In [ ]:
#@title

drive.mount('/content/gdrive')

FILES = os.listdir(FACEBOOK_DIR)
FILE_GROUPS = ('.edges', '.egofeat', '.feat', '.featnames', '.circles')
EDGES, EGO_FEAT, FEAT, FEAT_NAMES, CIRCLES = ([f for f in FILES if f.endswith(ext)]
                                              for ext in FILE_GROUPS)

G = (nx.read_edgelist(os.path.join(FACEBOOK_DIR, e)) for e in EDGES)
G = nx.compose_all(G)

# For simplicity, only consider the largest connected component.
largest_cc = max(nx.connected_components(G), key=len)
G = G.subgraph(largest_cc).copy()

pos = nx.spring_layout(G)

In [ ]:
#@title

print('Indivíduos:', len(G))
print('Relacionamentos:', len(G.edges))

In [ ]:
#@title definindo algumas funções úteis

def show_contamination_graph(G, pos, contaminated, verbose=1, ax=None):
    if verbose:
        print('population:', len(G))
        print(f'contaminated: {len(contaminated)} ({len(contaminated) / len(G):.2%})')

    nx.draw_networkx_nodes(G, pos,
                           node_color=~np.isin(G.nodes, contaminated),
                           node_size=10,
                           alpha=.8,
                           cmap=plt.cm.Set1,
                           ax=ax)
    nx.draw_networkx_edges(G, pos, width=1, alpha=.1, ax=ax)
    plt.axis('off')

def show_contamination_progress(G, pos, contaminated, verbose=1, ax=None):
    COLUMNS = 7

    for day, c in enumerate(contaminated):
        infection_rate = len(c) / len(G)

        plt.subplot(ceil(len(contaminated) / COLUMNS), COLUMNS, day + 1, title=f'Day {day} {infection_rate:.0%}')
        show_contamination_graph(G, pos, list(c), verbose=0)

def random_uniform_sample(G, rate):
  return np.asarray(G.nodes)[np.random.rand(len(G)) <= rate]

### Situação inicial de contaminação

In [ ]:
#@title

CONTAMINATED_RATE = .01
print('taxa de contaminados:', CONTAMINATED_RATE)

contaminated_0 = random_uniform_sample(G, CONTAMINATED_RATE)

show_contamination_graph(G, pos, contaminated_0)

## Modelo de infecção instantânea

Neste modelo, um indivíduo infectado imediatamente infecta seus vizinhos em um único *epoch*.

### Exibindo extensão da infecção

Neste primeiro cenário, os indivíduos continuam suas rotinas de interação normalmente.

In [ ]:
#@title

def expand_contamination(G, contaminated):
    contamination_trees = [nx.algorithms.dfs_tree(G, c) for c in contaminated]
    return (nx.compose_all(contamination_trees)
              .nodes)

contaminated_1 = expand_contamination(G, contaminated_0)

show_contamination_graph(G, pos, contaminated_1)

### Indivíduos aleatórios estão executando *home-officing*

According to
[IBGE](https://biblioteca.ibge.gov.br/visualizacao/livros/liv101694_informativo.pdf),
5.2% of the working class would work from home in 2018. Let's up-top that with
10% now that the pandemic is here.

In [ ]:
#@title

def expand_contamination_ho(G, contaminated, home_officing):
    G_nho = G.copy()
    G_nho.remove_nodes_from(home_officing)

    contaminated_not_ho = contaminated[~np.isin(contaminated, home_officing)]
    if len(contaminated_not_ho):
      contaminated_not_ho = expand_contamination(G_nho, contaminated_not_ho)
    else:
      contaminated_not_ho = []

    return list(set(contaminated) | set(contaminated_not_ho))


def experiment(home_office_rate, contaminated_rate):
    home_officing_0 = random_uniform_sample(G, home_office_rate)
    contaminated_0 = random_uniform_sample(G, contaminated_rate)
    contaminated_1 = expand_contamination_ho(G, contaminated_0, home_officing_0)
    
    show_contamination_graph(G, pos, contaminated_1)

In [ ]:
experiment(home_office_rate=0.1,
           contaminated_rate=0.01)

In [ ]:
experiment(home_office_rate=0.2,
           contaminated_rate=0.01)

In [ ]:
experiment(home_office_rate=0.5,
           contaminated_rate=0.01)

In [ ]:
experiment(home_office_rate=0.9,
           contaminated_rate=0.01)

## Modelo de infecção progressiva

Neste modelo, epochs (em dias) são executados e a infeção cobre a população progressivamente, respeitando o grafo de relacionamentos *Facebook Circles* e dois fatores:

- $E$: a probabilidade de um indivíduo se encontrar com um de seus vizinhos
- $p$: a probabilidade de que um encontro se torne uma infecção

O experimento conta também com algumas variáveis de controle. Elas estão listadas abaixo juntamente com seus valores padrões.
- `OUT_FOR_GROCERIES_PR`: probabilidade de um indivíduo sair ao supermercado (1/7)
- `INFECTIOUS_AFTER_DAYS`: número (em dias após contaminação) em que uma pessoa começa a ser infecciosa (3)
- `INFECTIOUS_FOR_DAYS`: número (em dias) em que uma pessoa é infecciosa (14)
- `CONTAMINATED_PR`: probabilidade inicial de um indivíduo da população estar contaminado (1%)
- `HOME_OFFICING_PR`: probabilidade inicial de um indivíduo estar fazendo *home-officing* (10%)

$N_d = E \cdot p \cdot N_d$  
$N_{d+1} = N_d + E \cdot p \cdot N_d \implies N_{d+1} = (1 + E\cdot p) N_d$

[1] https://www.youtube.com/watch?v=Kas0tIxDvrg

In [ ]:
def experiment():
    contaminated_0 = random_uniform_sample(G, CONTAMINATED_PR)
    home_officing_0 = random_uniform_sample(G, HOME_OFFICING_PR)
    cs, infs, c = expand_contamination(G, contaminated_0, home_officing_0, days=DAYS)

    print('Initial state:')
    print(f'  contaminated pr: {CONTAMINATED_PR:.2%}')
    print(f'  home-officing pr: {HOME_OFFICING_PR:.2%}')
    print(f'contaminated after {DAYS} days: {cs[-1]} ({cs[-1] / len(G):.0%} of {len(G)})')

    d = pd.DataFrame({ 'day': np.arange(DAYS), 'contaminated': cs, 'infectous': infs }).melt(id_vars=['day'])

    plt.figure(figsize=(16, 6))
    plt.subplot(121)
    sns.lineplot(data=d, x='day', y='value', hue='variable').set(ylim=(0, len(G)))
    
    plt.subplot(122)
    show_contamination_graph(G, pos, np.asarray(list(G.nodes))[c >= 0], verbose=0);

In [ ]:
def expand_contamination(G, c, ho, days=1):
    knows = np.asarray(nx.to_numpy_matrix(G)).astype(bool) # know each other
    ho = np.isin(G.nodes, ho)              # is home-officing
    c = np.isin(G.nodes, c).astype(float)  # contaminated for # days 
    c[c == 0] = -np.inf

    cs = []
    infs = []

    for day in range(days):
        left_home = (~ho | (rand(len(G)) <= OUT_FOR_GROCERIES_PR))
        infectious = ((c >= INFECTIOUS_AFTER_DAYS) &
                      (c < INFECTIOUS_AFTER_DAYS + INFECTIOUS_FOR_DAYS) &
                      left_home)

        c_day = ((knows[infectious, :] &
                  left_home.reshape(1, -1) &
                  (rand(infectious.sum(), len(G)) <= MEETING_PR * INFECTION_PR))
                 .any(axis=0)) # any of the acquaintances infected them
        
        c[c_day] = np.maximum(0, c[c_day]) # it is now infected
        c += 1                             # the day is over
        
        cs.append((c >= 0).sum())
        infs.append(infectious.sum())

    return cs, infs, c

In [ ]:
DAYS = 365

MEETING_PR = 0.1
INFECTION_PR = 0.05
OUT_FOR_GROCERIES_PR = 1 / 7
# TODO: LEAVES_HOME_IF_INFECTOUS_PROBA = .1

INFECTIOUS_AFTER_DAYS = 3
INFECTIOUS_FOR_DAYS = 14

CONTAMINATED_PR=.01
HOME_OFFICING_PR=.1

experiment()

In [ ]:
HOME_OFFICING_PR=.3

experiment()

In [ ]:
HOME_OFFICING_PR=.5

experiment()

Segundo [pesquisa do IBGE e artigo da Folha](https://www1.folha.uol.com.br/cotidiano/2020/04/28-dos-brasileiros-nao-fazem-isolamento-contra-coronavirus-diz-datafolha.shtml),
22% dos brasileiros não estão em quarentena.

In [ ]:
HOME_OFFICING_PR=.78

experiment()